In [ ]:
#Import and declare TIC

%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import lightkurve as lk
from scipy.interpolate import make_interp_spline as spline
import scipy.signal as signal
import matplotlib
from modules import *
from scipy.optimize import curve_fit

plt.style.use('seaborn-v0_8-darkgrid')

TIC = 'TIC 61285257'

In [ ]:
#Download the data and stitch it together

search_results = lk.search_lightcurve(TIC, cadence='short', author='SPOC')
print(search_results)

#Enter the data index to be used                                                                                        [INPUT NEEDED]
use_till = 5
start = 0
end = 1
lcs = []
for i in range(start, end):
    try:
        lcs.append(search_results[i].download(quality_bitmask='hardest'))
    except:
        pass

In [ ]:
time_interval = 1

times = [np.array(x.time.btjd) for x in lcs]
cadence_multiplier = 86400/(np.median(np.diff(times[0][0:100])) * 86400).round()
olds = [x[0] for x in times]
times = [(x - x[0]) * cadence_multiplier for x in times]
times = [x.round(0) for x in times]
fluxes = [np.array(x.flux, dtype=float) for x in lcs]

min_times = [x[0] for x in times]
max_times = [x[-1] for x in times]
full_times = [np.arange(x, y + time_interval, time_interval) for x, y in zip(min_times, max_times)]
savgols = []

for i in range(len(times)):
    #Filling in missing times with nans, and then interpolating gaps with staright lines
    df = pd.DataFrame({'Time':times[i], 'Flux':fluxes[i]})
    df.set_index('Time', inplace=True)
    df = df.reindex(full_times[i])
    df['Flux'] = df['Flux'].interpolate(method='linear')
    df.reset_index(inplace=True)
    df.rename(columns={'index':'Time'}, inplace=True)
    df['Time'] = (df['Time'] / cadence_multiplier) + olds[i] + 2457000
    df.dropna(inplace=True)
    df.reset_index(inplace=True)

    #Interpolating using spline
    time_smooth = np.linspace(df['Time'].min(), df['Time'].max(), len(df['Time']) * 4)
    spl = spline(df['Time'], df['Flux'], k=3)
    flux_smooth = spl(time_smooth)

    #Applying the savgol filter
    del df
    df = pd.DataFrame({'time':time_smooth, 'flux':flux_smooth})
    df = apply_savgol_filter(df['time'], df['flux'])
    savgols.append(df)

In [ ]:
corrected_df = pd.concat(savgols, ignore_index=True)
corrected_df.info()


def sin_fit(x, amp, freq, phase, offset):
    return amp * np.sin(2 * np.pi * freq * (x - phase)) + offset

time = corrected_df['time'].values
flux = corrected_df['flux'].values

s = sin_fit(time, 20, 3.33, 0, -10)

plt.figure(figsize=(10, 5))
plt.plot(time, flux, 'k', label='Light Curve Data', linewidth = 0.4)
plt.plot(time, s, 'r-', label='Sinusoidal Fit', linewidth = 0.8)
plt.xlabel('Time')
plt.ylabel('Flux')
plt.legend()
plt.show()

In [ ]:
691-611

In [ ]:
0.763662-0.7630